# IEEE-CIS Fraud Detection Part 01: Tìm hiểu bài toán + Phân tích dữ liệu + Feature Engineering

# **Nguyễn Huy Sơn - 18021102 - Học Máy**

# 1. Tìm hiểu bài toán

## 1.1 Bài toán
Bài toán nhận dữ liệu đến từ các giao dịch thương mại điện tử trong thực tế của Vesta - một công ty cung cấp dịch vụ thanh toán hàng đầu và chứa một loạt các tính năng từ loại thiết bị đến tính năng của sản phẩm.
Nhiệm vụ của cuộc thi này là xây dựng các mô hình học máy để dự đoán xác suất để một giao dịch trực tuyến là gian lận (isFraud).

 ## 1.2 Mục tiêu
Xác định mục tiêu nghiệp vụ chính của việc tìm kiếm các giao dịch gian lận:
* Dự đoán xác suất giao dịch gian lận
* Không nên dự đoán các giao dịch gian lận là không gian lận và ngược lại. Vì vậy, cần chú ý cả precision và recall.
* Xây dựng các mô hình dự đoán để giải quyết bài toán phân loại nhị phân -> giảm thiểu cả *false negatives (FN)* và *false positives (FP)*.
* Chú ý các giao dịch lớn, vì chúng có thể gây ra tổn thất lớn hơn nếu sai sót. Đặt ngưỡng xác suất thấp hơn -> chặt chẽ hơn để phân loại giao dịch gian lận khi liên quan đến số tiền giao dịch cao hơn.
* Hiểu những yếu tố/ đặc trưng nào mang tính dự đoán cao hơn để phát hiện gian lận.

# 2. Nhận định bài toán

## 2.1 Về dữ liệu

Dữ liệu được chia thành hai tệp nhận dạng và giao dịch, được kết hợp với mỗi tệp bằng **TransactionID** để *train* và *test* (có 4 tệp như input và 1 tệp sample_submission).
Điều quan trọng cần lưu ý là tất cả các giao dịch không có thông tin nhận dạng tương ứng.
- Tập định danh Train chứa 144233 hàng và 41 tính năng.
- Tập giao dịch Train chứa 590540 hàng và 394 tính năng.

### Mô tả

**Data**:  
- *TransactionID*: Id của giao dịch.
- *identity* and *transaction*, được join bởi TransactionID. Không phải tất cả các giao dịch đều có *identity information*.
- *TransactionDT* là một bộ đếm thời gian từ một ngày giờ tham chiếu nhất định (không phải là một dấu thời gian thực tế) - (not an actual timestamp).

**Categorical Features - Transaction**:
* TransactionAmt : số tiền thanh toán giao dịch bằng USD  
* ProductCD : product CODE 
* card1 - card6: thông tin thẻ thanh toán, chẳng hạn như loại thẻ, loại thẻ, ngân hàng phát hành, quốc gia, v.v..
* addr1, addr2
* P_emaildomain : miền email của người mua và người nhận (purchaser and recipient)
* R_emaildomain
* M1 - M9
* C1-C14: đếm, chẳng hạn như có bao nhiêu địa chỉ được tìm thấy có liên quan đến thẻ thanh toán, v.v. *The actual meaning is masked*.
* D1-D15: timedelta - khoảng cách bao nhiêu ngày giữu các giao dịch trước, etc.
* M1-M9: match - ví dụ tên trên card và address, etc.
* Vxxx: Vesta đã thiết kế các tính năng phong phú bao gồm xếp hạng, đếm và các quan hệ thực thể khác.

**Categorical Features - Identity**:
* DeviceType: Loại thiết bị được sử dụng cho giao dịch.
* DeviceInfo: Thông tin thêm về thiết bị được sử dụng.
* id_1 - id_38: Thông tin kết nối mạng, thông tin trình duyệt, v.v. (id 12–38 là thông tin phân loại).
Xem thêm tham khảo tại: https://www.kaggle.com/c/ieee-fraud-detection/discussion/101203

## 2.2 Xác định bài toán học máy

- Bài toán phân loại nhị phân (2 lớp): giao dịch gian lận được ký hiệu bằng 1, giao dịch hợp pháp được ký hiệu bằng 0. 
- Được đánh giá trên **Area under the ROC curve** giữa xác suất dự đoán và mục tiêu.

### Mục tiêu và khó khăn:
- Mục tiêu: Tối đa hóa *area under the ROC curve* giữa xác suất dự đoán và mục tiêu.
- Mô hình phải đưa ra đầu ra có xác suất.
- Yêu cầu về độ trễ thấp
- Giảm thiểu cả *false positive* và *false negative*

# 3. Phân tích và xử lý dữ liệu

In [ ]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.sparse
import gc
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.metrics import roc_auc_score
import warnings 
from lightgbm import LGBMClassifier
warnings.filterwarnings('ignore')

### Đọc file và hợp nhất trên cả *train* và *test* dựa vào cột *TransactionID*

In [ ]:
identity_train = pd.read_csv("/kaggle/input/ieee-fraud-detection/train_identity.csv")
identity_test = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_identity.csv')
test_transaction = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_transaction.csv')
train_transaction = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv')

In [ ]:
# hợp nhất data dựa theo thuộc tính TransactionID 
train = pd.merge(train_transaction,identity_train, on='TransactionID', how='left')
test = pd.merge(test_transaction,identity_test, on='TransactionID', how='left')

In [ ]:
train

In [ ]:
test

#### Nhận xét sơ bộ về dữ liệu:
- Cột ***isFraud*** là mục tiêu cần xác định.
- ***TransactionDT*** là thời gian (Không phải thời gian thực tế mà từ một tham chiếu cụ thể) giao dịch xảy ra (tính bằng sec.) Dữ liệu trên tập test lại có giá trị lớn hơn nhiều nên ***không overlap***. 

#### Kiểm tra lượng giá trị null trên dữ liệu
Ta sẽ quan sát số lượng giá trị null có trong tất cả các feature và kiểu dữ liệu của nó thông qua hàm sau:

In [ ]:
# train data
for i in train.columns:
    print(i,'NaN values present is:',train[i].isnull().sum(),end=" ")
    print('type is :',train[i].dtypes,end=" ")
    print('% of NaN values:',np.round(train[i].isnull().sum()/train.shape[0]*100,2))

In [ ]:
# test data
for i in test.columns:
    print(i,'NaN values present is:',test[i].isnull().sum(),end=" ")
    print('type is :',test[i].dtypes,end=" ")
    print('% of NaN values:',np.round(test[i].isnull().sum()/test.shape[0]*100,2))

**Nhận xét:**
Nhiều feature chứ lượng lớn giá trị null

-> cần thực hiện loại bỏ các giá trị bằng các phương pháp thay thế (được thực hiện ở dưới đây).

#### Giảm memory
Kiểu dữ liệu chứa nhiều int64, int32, float64, v.v.

-> Cần giảm để tránh bị tràn memory.

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: 
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
# giải phóng data frame không cần thiết
del train_transaction,test_transaction,identity_train,identity_test
gc.collect()

In [ ]:
# giảm memory
train=reduce_mem_usage(train)
test=reduce_mem_usage(test)
print('training set shape:', train.shape)
print('test set shape:', test.shape)

Tiếp theo, bắt đầu thực hiện phân tích dữ liệu về các feature được cung cấp và tìm sự tương quan dữ liệu giữa tập train và tập test.

In [ ]:
sns.countplot(train['isFraud'])
plt.show()
print('Dữ liệu chứa ',np.round(train[train['isFraud']==1].shape[0]/train.shape[0]*100,2),'% giao dịch gian lận.')
print('Dữ liệu chứa ',np.round(train[train['isFraud']==0].shape[0]/train.shape[0]*100,2),'% giao dịch hợp pháp.')

Trên đây là đồ thị về sự phân bố dữ liệu của 2 nhãn 0 & 1 (isFraud).  

-> Dễ dàng nhận thấy sự **mất cân bằng dữ liệu** - một vấn đề phức tạp cho bài toán phân loại.

Tạo một hàm 'describe' để thuận tiện việc mô tả các tính chất của mỗi feature:

In [ ]:
def describe(datatrain,datatest,feature):
    d = pd.DataFrame(columns=[feature,'Train','TrainFraud','TrainLegit','Test'])
    d[feature] = ['count','mean','std','min','25%','50%','75%','max','unique','NaN','NaNshare']
    for i in range(0,8):
        d['Train'].iloc[i] = datatrain[feature].describe().iloc[i]
        d['TrainFraud'].iloc[i]=datatrain[datatrain['isFraud']==1][feature].describe().iloc[i]
        d['TrainLegit'].iloc[i]=datatrain[datatrain['isFraud']==0][feature].describe().iloc[i]
        d['Test'].iloc[i]=datatest[feature].describe().iloc[i]
    d['Train'].iloc[8] = len(datatrain[feature].unique())
    d['TrainFraud'].iloc[8]=len(datatrain[datatrain['isFraud']==1][feature].unique())
    d['TrainLegit'].iloc[8]=len(datatrain[datatrain['isFraud']==0][feature].unique())
    d['Test'].iloc[8]=len(datatest[feature].unique())
    d['Train'].iloc[9] = datatrain[feature].isnull().sum()
    d['TrainFraud'].iloc[9] = datatrain[datatrain['isFraud']==1][feature].isnull().sum()
    d['TrainLegit'].iloc[9] = datatrain[datatrain['isFraud']==0][feature].isnull().sum()
    d['Test'].iloc[9]=datatest[feature].isnull().sum()
    d['Train'].iloc[10] = datatrain[feature].isnull().sum()/len(datatrain)
    d['TrainFraud'].iloc[10] = datatrain[datatrain['isFraud']==1][feature].isnull().sum()/len(datatrain[datatrain['isFraud']==1])
    d['TrainLegit'].iloc[10] = datatrain[datatrain['isFraud']==0][feature].isnull().sum()/len(datatrain[datatrain['isFraud']==0])
    d['Test'].iloc[10]=datatest[feature].isnull().sum()/len(datatest)
    return d

<h4> Phân tích về feature TransactionDT trên 2 tập train và test </h4>

**TransactionDT** là một trong những tính năng quan trọng nhất trong tập dữ liệu, liên quan đến thời gian (tính bằng giây/sec.)

In [ ]:
transactionDTDescribe = describe(train,test,'TransactionDT')

In [ ]:
transactionDTDescribe

In [ ]:
sns.distplot(train['TransactionDT'],kde=False)
sns.distplot(test['TransactionDT'],kde=False)
plt.legend(['train','test'])

**Nhận xét:**
- Khoảng thời gian trên feature của 2 bộ test và train là rời rạc -> thực hiện phân tách theo cụm thời gian.
- Một điều chú ý là phần đầu của train có thời gian giao dịch cao, phần cuối của test cũng vậy.

Ta sẽ tạo ra 2 features mới về thời gian: 'hour' và 'day' thay vì tính bằng 'sec' như cũ.

In [ ]:
train['day'] = (train['TransactionDT']//(3600*24)-1)%7
test['day'] = (test['TransactionDT']//(3600*24)-1)%7

Tiếp theo, phân tích một số đặc điểm từ thuộc tính này:

In [ ]:
train_day = (train.groupby(['isFraud'])['day']
                     .value_counts(normalize=True)
                     .rename('percentage')
                     .mul(100)
                     .reset_index()
                     .sort_values('day'))
sns.barplot(x="day", y="percentage", hue="isFraud", data=train_day)

**Nhận xét:**
Trong cả 7 ngày, giao dịch hợp pháp và gian lận đã xảy ra với tỷ lệ (phần trăm) gần như bằng nhau

In [ ]:
train['hour'] = (train['TransactionDT']//(3600))%24
test['hour'] = (test['TransactionDT']//(3600))%24

In [ ]:
train_hour = (train.groupby(['isFraud'])['hour']
                     .value_counts(normalize=True)
                     .rename('percentage')
                     .mul(100)
                     .reset_index()
                     .sort_values('hour'))
sns.barplot(x="hour", y="percentage", hue="isFraud", data=train_hour)

**Nhận xét:**

Từ biểu đồ, có thể nhận thấy rằng giờ từ 4 đến 9 có tỷ lệ giao dịch gian lận nhiều hơn và từ 9h đến 18h tỷ lệ giao dịch hợp pháp nhiều hơn.

In [ ]:
# Tạo hour feature - mang một chút tính cảnh báo dựa vào các phân tích trên
def hourFeature(hour):
    if hour>3 and hour < 11:
        return "highalert"
    if hour ==11 or hour==18:
        return "lowalert"
    if hour==2 or hour==3 or hour==23:
        return "mediumalert"
    else:
        return "noalert"

In [ ]:
train['alertFeature'] = train['hour'].apply(hourFeature)
test['alertFeature'] = test['hour'].apply(hourFeature)

<h4> Phân tích feature TransactionAmt </h4>

**TransactionAmt** biểu thị cho số tiền thanh toán giao dịch bằng USD

In [ ]:
transactionAmtDescribe = describe(train,test,'TransactionAmt')

In [ ]:
transactionAmtDescribe

**Nhận xét:**
* Từ bảng mô tả trên, ta nhận thấy không có giá trị null nào trong cả tập dữ liệu train và test. 
* Đặc điểm đáng chú ý là có tới 75% các giá trị dưới 125 nhưng đột nhiên nó tăng lên khoảng 32K trong train và 10K trong test. -> Có thể có một số ngoại lệ ở đó.

Ở đây ta sử dụng phương pháp **np.percentile** trong numpy (quan sát thấy rằng 99,99 giá trị phân vị cho cả train và test đều nằm trong phạm vi 6K). 

-> Tất cả các hàng có giá trị TransactionAmt lớn hơn 6K sẽ bị xóa trên tập train để huấn luyện bớt nhiễu hơn. 

Feature này có **phân phối lệch** -> cần tạo ra một tính năng khác có tên là **LogTransactionAmt** - mang tính giống như phân phối bình thường bây giờ.

In [ ]:
l=[99.9,99.91,99.92,99.93,99.94,99.95,99.96,99.97,99.98,99.99]
for i in l:
    print('train',np.percentile(train['TransactionAmt'],i))
    print('test',np.percentile(test['TransactionAmt'],i))

- **Không có sự khác biệt các giá trị** trong phân vị 99,99.

-> Trong tập train, thuộc tính TrainsactionAmt **có vấn đề xấu**.

In [ ]:
# thử sử dụng ngưỡng 10000 trở lên:
train[train['TransactionAmt']>10000]

- Chỉ có 2 giá trị > 10.000 mà còn bị trùng lặp nhiều 

-> Loại bỏ

In [ ]:
train = train[train['TransactionAmt']<10000]

In [ ]:
# lấy giá trị log của TransactionAmt để tạo ra thuộc tính mới LogTransactionAmt
train['LogTransactionAmt'] = np.log(train['TransactionAmt'])
test['LogTransactionAmt'] = np.log(test['TransactionAmt'])

In [ ]:
plt.figure(figsize=(9,6))
plt.subplot(1,2,1)
sns.distplot(train[train['isFraud']==0]['LogTransactionAmt'])
sns.distplot(train[train['isFraud']==1]['LogTransactionAmt'])
plt.legend(['legit','fraud'])
plt.title('Train')
# plt.subplot(1,2,2)
# sns.distplot(test['LogTransactionAmt'])

**Nhận xét:**
* Có trùng lặp nhưng logTransactionAmt sau 5 và trước 3 có tần suất gian lận cao hơn.
* logTransactionAmt từ 3 đến 5 có cơ hội cao hơn là giao dịch hợp pháp

<h4> Phân tích về feature ProductCD trên train và test </h4>

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
train_ProductCD = (train.groupby(['isFraud'])['ProductCD']
                     .value_counts(normalize=True)
                     .rename('percentage')
                     .mul(100)
                     .reset_index()
                     .sort_values('ProductCD'))
sns.barplot(x="ProductCD", y="percentage", hue="isFraud", data=train_ProductCD)
plt.subplot(1,2,2)
test_ProductCD =test['ProductCD'].value_counts(normalize=True).mul(100).rename('percentage')\
.reset_index()
sns.barplot(x="index", y="percentage", data=test_ProductCD)

**Nhận xét:**
* Theo đồ thị, ta thấy nếu ProductCD là "C" thì có 40% khả năng đó là gian lận.
* Đối với H, R, S khả năng là tỷ lệ gian lận cao.

#### Phân tích feature card3 

In [ ]:
plt.figure(figsize=(11,6))
plt.subplot(1,2,1)
sns.distplot(train[(train['isFraud']==0) & (~train['card3'].isnull())]['card3'])
sns.distplot(train[(train['isFraud']==1) & (~train['card3'].isnull())]['card3'])
plt.legend(['Legit','Fraud'])
plt.title('Train')
plt.subplot(1,2,2)
sns.distplot(test[~test['card3'].isnull()]['card3'])
plt.title('Test')

**Nhận xét:**
- Nhận thấy giá trị lớn hơn 150, xác suất giao dịch là gian lận sẽ tăng lên.
- Ta sẽ tạo một tính năng khác như nếu giá trị lớn hơn 150 thì đánh dấu nó là T (có nghĩa là gian lận) nếu không thì F.

In [ ]:
def card3Values(val):
    if val==np.nan:
        return val
    else:
        if val > 150:
            return 'T'
        else:
            return 'F'

In [ ]:
train['card3Values'] = train['card3'].apply(card3Values)

In [ ]:
test['card3Values'] = test['card3'].apply(card3Values)

<h4> Phân tích feature card4 </h4>

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
train_card4 = (train[~train['card4'].isnull()].groupby(['isFraud'])['card4']
                     .value_counts(normalize=True)
                     .rename('percentage')
                     .mul(100)
                     .reset_index()
                     .sort_values('card4'))
sns.barplot(x="card4", y="percentage", hue="isFraud", data=train_card4)
plt.title('Train')
plt.subplot(1,2,2)
test_card4 =test[~test['card4'].isnull()]['card4'].value_counts(normalize=True).mul(100).rename('percentage')\
.reset_index()
sns.barplot(x="index", y="percentage", data=test_card4)
plt.title('Test')

**Nhận xét:**
- Hầu hết các giao dịch diễn ra thông qua loại thẻ visa, còn thẻ mastercard cũng tỉ lệ khá cao.
- Trong các giao dịch gian lận, việc sử dụng thẻ được thấy nhiều hơn so với giao dịch hợp pháp.


<h4> Phân tích feature card6 </h4>

**Card6** cho biết liệu giao dịch được thực hiện thông qua thẻ tín dụng hay thẻ ghi nợ hoặc thẻ tính phí.

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
train_card6 = (train[~train['card6'].isnull()].groupby(['isFraud'])['card6']
                     .value_counts(normalize=True)
                     .rename('percentage')
                     .mul(100)
                     .reset_index()
                     .sort_values('card6'))
sns.barplot(x="card6", y="percentage", hue="isFraud", data=train_card6)
plt.title('Train')
plt.subplot(1,2,2)
test_card6 =test[~test['card6'].isnull()]['card6'].value_counts(normalize=True).mul(100).rename('percentage')\
.reset_index()
sns.barplot(x="index", y="percentage", data=test_card6)
plt.title('Test')

**Nhận xét:**
- Thẻ ghi nợ hoặc thẻ tín dụng và thẻ tính phí gần như bằng 0% trong tập dữ liệu.
- Cho phép kết hợp thẻ thành danh mục thẻ mới.
- Chủ thẻ tín dụng có xu hướng giao dịch gian lận nhiều hơn chủ thẻ ghi nợ.

In [ ]:
def replaceToOther(value):
    if value==np.nan:
        return value
    if value=='debit or credit' or value=='charge card':
        return 'debit'
    else:
        return value

In [ ]:
train['card6'] = train['card6'].apply(replaceToOther)
test['card6'] = test['card6'].apply(replaceToOther)

<h4> Phân tích features : addr1,addr2  </h4>

- addr1 là khu vực thanh toán (mã zip)
- addr2 là quốc gia thanh toán

Cả hai addr đều dành cho người mua

<h4> Phân tích feature P_emaildomain, R_emaildomain</h4>

**P_emaildomain** đề cập đến email của người mua và **R_emaildomain** đề cập đến email của người nhận

In [ ]:
train['P_emaildomain'].value_counts()[:10]

- Mail: jp (nhật bản), fr (pháp), de (đức), uk (Anh), mx (mexico), es (tây ban nha)

-> Tạo thêm 1 feature mới từ các đặc điểm này.

In [ ]:
def returnfirst(email):
    return email.split(".")[0]

Đầu tiên, ta tách phần đầu tiên khỏi phần thứ hai của thư. 

Từ phần đầu tiên, một feature mới sẽ được tạo và từ phần thứ hai, một feature mới khác sẽ được tạo. 

Phần thứ hai bao gồm net, com, de , es, jp, fr, uk, mx, es ... 

Phần đầu tiên bao gồm msn, hotmail, outlook, gmail, me, icloud, v.v. Tất cả phần đầu thuộc về một số công ty, như msn, hotmail , outlook - Microsoft, icloud - Apple.

In [ ]:
train['first'] = train[~train['P_emaildomain'].isnull()]['P_emaildomain'].apply(returnfirst)

In [ ]:
test['first'] = test[~test['P_emaildomain'].isnull()]['P_emaildomain'].apply(returnfirst)

In [ ]:
train_email = (train.groupby(['isFraud'])['first']
                     .value_counts(normalize=True)
                     .rename('percentage')
                     .mul(100)
                     .reset_index()
                     .sort_values('first'))

In [ ]:
sns.barplot(x="first", y="percentage", hue="isFraud", data=train_email)
plt.xticks(rotation=90)

- Tạo ra 2 feature: tên mail và extension tương ứng.
- Kết hợp các mail tương đồng (cùng tên miền) lại.

In [ ]:
emails = {'gmail': 'google', 'att.net': 'att', 'twc.com': 'spectrum',
'scranton.edu': 'other', 'optonline.net': 'other', 'hotmail.co.uk': 'microsoft', 'comcast.net': 'other', 'yahoo.com.mx': 'yahoo', 'yahoo.fr'
: 'yahoo', 'yahoo.es': 'yahoo', 'charter.net': 'spectrum', 'live.com':
'microsoft', 'aim.com': 'aol', 'hotmail.de': 'microsoft', 'centurylink.net': 'centurylink', 
'gmail.com': 'google', 'me.com': 'apple', 'earthlink.net': 'other', 'gmx.de': 'other', 
'web.de': 'other', 'cfl.rr.com': 'other', 'hotmail.com': 'microsoft', 'protonmail.com': 'other', 
'hotmail.fr': 'microsoft', 'windstream.net': 'other', 'outlook.es': 'microsoft', 
'yahoo.co.jp': 'yahoo', 'yahoo.de': 'yahoo', 'servicios-ta.com': 'other', 'netzero.net': 'other', 
'suddenlink.net': 'other', 'roadrunner.com': 'other', 'sc.rr.com': 'other', 'live.fr': 'microsoft', 
'verizon.net': 'yahoo', 'msn.com': 'microsoft', 'q.com': 'centurylink',
'prodigy.net.mx': 'att', 'frontier.com': 'yahoo', 'anonymous.com': 'other', 'rocketmail.com': 'yahoo', 
'sbcglobal.net': 'att', 'frontiernet.net': 'yahoo', 'ymail.com': 'yahoo', 'outlook.com': 'microsoft', 
'mail.com': 'other', 'bellsouth.net': 'other', 'embarqmail.com': 'centurylink', 
'cableone.net': 'other', 'hotmail.es': 'microsoft', 'mac.com':
'apple', 'yahoo.co.uk': 'yahoo', 'netzero.com': 'other', 'yahoo.com':
'yahoo', 'live.com.mx': 'microsoft', 'ptd.net': 'other', 'cox.net': 'other', 'aol.com': 'aol', 
'juno.com': 'other', 'icloud.com': 'apple'}
us_emails = ['gmail', 'net', 'edu']
for c in ['P_emaildomain', 'R_emaildomain']:
    train[c + '_bin'] = train[c].map(emails)
    test[c + '_bin'] = test[c].map(emails)
    train[c + '_suffix'] = train[c].map(lambda x: str(x).split('.')[-1])
    test[c + '_suffix'] = test[c].map(lambda x: str(x).split('.')[-1])
    train[c + '_suffix'] = train[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')
    test[c + '_suffix'] = test[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')

Ta đã tạo 2 tính năng là ‘P_emaildomain_bin’ và ‘P_emaildomain_prefix’ :
* P_emaildomain_bin : tên công ty cho email đó 
* P_emaildomain_prefix : extension của email đó 

Tương tự là R_emaildomain_perfix và R_emaildomain_bin.

In [ ]:
# bỏ đi "first" 
train.drop(['first'],axis=1,inplace=True)

<h4> Phân tích một số feature C </h4>

In [ ]:
plt.figure(figsize=(11,6))
plt.subplot(1,2,1)
sns.scatterplot(x="TransactionDT",y="C1",hue="isFraud",data=train,alpha=0.7,hue_order=[0,1])
plt.title('Train')
plt.subplot(1,2,2)
sns.scatterplot(x="TransactionDT",y="C1",data=test[~test['C1'].isnull()])
plt.title('Test')

Ta có thể thấy giá trị khoảng 2000 thì không sao nhưng lên trên 3000 thì xuất hiện dị thường.

-> Loại bỏ tất cả các hàng có giá trị> 2000.

In [ ]:
train = train[train['C1']<=2000]

In [ ]:
# visualize C2
plt.figure(figsize=(11,6))
plt.subplot(1,2,1)
sns.scatterplot(x="TransactionDT",y="C2",hue="isFraud",data=train,alpha=0.7,hue_order=[0,1])
plt.title('Train')
plt.subplot(1,2,2)
sns.scatterplot(x="TransactionDT",y="C2",data=test[~test['C2'].isnull()])
plt.title('Test')

Cả dữ liệu train và test đều có giá trị ngoại lệ.

-> Loại bỏ giá trị trên 2000 trong C2.

In [ ]:
train = train[train['C2']<=2000]

**Feature C12**

In [ ]:
c12des = describe(train,test,'C12')
c12des

Không có giá trị null trong C12 trên train, và chỉ có 3 giá trị null trong C12 test. 

75% giá trị trong C12 là 0 (train). Và giá trị tối đa là 1187 có nghĩa là có một số ngoại lệ cần phải loại bỏ.

In [ ]:
# Vẽ một biểu đồ phân tán lấy trục x làm TransactionDT và trục y là C12.
plt.figure(figsize=(11,6))
plt.subplot(1,2,1)
sns.scatterplot(x="TransactionDT",y="C12",hue="isFraud",data=train,alpha=0.7,hue_order=[0,1])
plt.title('Train')
plt.subplot(1,2,2)
sns.scatterplot(x="TransactionDT",y="C12",data=test[~test['C12'].isnull()])
plt.title('Test')

**Nhận xét biểu đồ:**
* Giao dịch gian lận có giá trị C12 nhỏ chủ yếu nằm trong khoảng 200. 

* Cả trên train và test đều nhận một số giá trị cực đoan trong thời gian đầu tiên của train và thời gian cuối cùng của test.

**Vấn đề gặp phải: Giá trị cực đoan (extreme values)**
* Không thể train mô hình tree-based sau một độ sâu nhất định bởi vì nó sẽ bắt các giá trị ngoại lai -> overfitting.
* Đối với Logistic regression cũng bị ảnh hưởng lớn.

In [ ]:
cor_c = train[['C1','C2','C3','C4','C5','C6','C7','C8','C9','C10','C11','C12','C13','C14','isFraud']]

In [ ]:
f = cor_c.corr()

In [ ]:
plt.figure(1,figsize=(18,18))
sns.heatmap(f,annot=True)

Ta có thể thấy rằng một số feature được kết hợp chặt chẽ với nhau: 'C1' và 'C2', 'C1' và 'C6', 'C1' và 'C14', 'C6' và 'C14', v.v.

-> **việc loại bỏ các feature tương quan cao này sẽ cải thiện acc hoặc giảm acc**.

In [ ]:
cor_c_test = test[['C1','C2','C3','C4','C5','C6','C7','C8','C9','C10','C11','C12','C13','C14']]

In [ ]:
f = cor_c_test.corr()

In [ ]:
plt.figure(1,figsize=(18,18))
sns.heatmap(f,annot=True)

- Trên test: giá trị của các 'C'  cũng Có tương quan cao.

<h4> Phân tích một số feature D</h4>

Các tính năng D liên quan đến tham chiếu thời gian.

**Feature D4**

In [ ]:
# visualize D4
plt.figure(figsize=(11,6))
plt.subplot(1,2,1)
sns.scatterplot(x="TransactionDT",y="D4",hue="isFraud",data=train[~train['D4'].isnull()],alpha=0.7,hue_order=[0,1])
plt.title('Train')
plt.subplot(1,2,2)
sns.scatterplot(x="TransactionDT",y="D4",data=test[~test['D4'].isnull()])
plt.title('Test')

In [ ]:
# nhiều giá trị <0 có tính ngoại lai -> loại bỏ
train[train['D4']<0].index

In [ ]:
train.drop([2947,   4210,   5264,   5501,   6057,   9120, 110695, 268153,
            444548, 444552, 445576, 446484, 455658, 456223, 473396],inplace=True)

**Feature D11:**

In [ ]:
#visualize D11
plt.figure(figsize=(11,6))
plt.subplot(1,2,1)
sns.scatterplot(x="TransactionDT",y="D11",hue="isFraud",data=train[~train['D11'].isnull()],alpha=0.7,hue_order=[0,1])
plt.title('Train')
plt.subplot(1,2,2)
sns.scatterplot(x="TransactionDT",y="D11",data=test[~test['D11'].isnull()])
plt.title('Test')

In [ ]:
# nhận thấy các giá trị <0 dị thường -> loại bỏ
train[train['D11']<0].index

In [ ]:
train.drop([3814, 4932, 358337, 359698, 442488],inplace=True)

**Feature D12:**

In [ ]:
plt.figure(figsize=(11,6))
plt.subplot(1,2,1)
sns.scatterplot(x="TransactionDT",y="D12",hue="isFraud",data=train[~train['D12'].isnull()],alpha=0.7,hue_order=[0,1])
plt.title('Train')
plt.subplot(1,2,2)
sns.scatterplot(x="TransactionDT",y="D12",data=test[~test['D12'].isnull()])
plt.title('Test')

**Nhận xét:**

Đối với D12, các tính năng hợp pháp và giao dịch gian lận hoàn toàn bị **overlapped**.

**Feature D14:**

In [ ]:
plt.figure(figsize=(11,6))
plt.subplot(1,2,1)
sns.scatterplot(x="TransactionDT",y="D14",hue="isFraud",data=train[~train['D14'].isnull()],alpha=0.7,hue_order=[0,1])
plt.title('Train')
plt.subplot(1,2,2)
sns.scatterplot(x="TransactionDT",y="D14",data=test[~test['D14'].isnull()])
plt.title('Test')

In [ ]:
# nhận thấy các giá trị <0 dị thường -> loại bỏ
train[train['D14']<0].index

In [ ]:
train.drop([4085,4097],inplace=True)

**Feature D15:**

In [ ]:
plt.figure(figsize=(11,6))
plt.subplot(1,2,1)
sns.scatterplot(x="TransactionDT",y="D15",hue="isFraud",data=train[~train['D15'].isnull()],alpha=0.7,hue_order=[0,1])
plt.title('Train')
plt.subplot(1,2,2)
sns.scatterplot(x="TransactionDT",y="D15",data=test[~test['D15'].isnull()])
plt.title('Test')

In [ ]:
# nhận thấy các giá trị <0 dị thường -> loại bỏ
train[train['D15']<0].index

In [ ]:
train.drop([3034, 7589, 13149, 442444, 442467],inplace=True)

**Nhận xét:**

Vì D chỉ timedelta -> không thể chứa "-ve" -> loại bỏ.

<h4> Phân tích một số feature M</h4>

In [ ]:
for i in ['M1','M2','M3','M4','M5','M6','M7','M8','M9']:
    print(train[(~train[i].isnull())&(train['isFraud']==0)][i].value_counts())
    print(train[(~train[i].isnull())&(train['isFraud']==1)][i].value_counts())
    print('----------------------------------------------------------------------')

**Với mỗi M sẽ tạo ra một biểu đồ hiển thị tỷ lệ phần trăm của cả giao dịch gian lận và hợp pháp cho mỗi hạng mục.**

In [ ]:
plt.figure(figsize=(18,18))
plt.subplot(3,3,1)
train_m1 = (train.groupby(['isFraud'])['M1']
                     .value_counts(normalize=True)
                     .rename('percentage')
                     .mul(100)
                     .reset_index()
                     .sort_values('M1'))
sns.barplot(x="M1", y="percentage", hue="isFraud", data=train_m1)
plt.title('TrainM1')
plt.subplot(3,3,2)
train_m2 = (train.groupby(['isFraud'])['M2']
                     .value_counts(normalize=True)
                     .rename('percentage')
                     .mul(100)
                     .reset_index()
                     .sort_values('M2'))
sns.barplot(x="M2", y="percentage", hue="isFraud", data=train_m2)
plt.title('TrainM2')
plt.subplot(3,3,3)
train_m3 = (train.groupby(['isFraud'])['M3']
                     .value_counts(normalize=True)
                     .rename('percentage')
                     .mul(100)
                     .reset_index()
                     .sort_values('M3'))
sns.barplot(x="M3", y="percentage", hue="isFraud", data=train_m3)
plt.title('TrainM3')
plt.subplot(3,3,4)
train_m4 = (train.groupby(['isFraud'])['M4']
                     .value_counts(normalize=True)
                     .rename('percentage')
                     .mul(100)
                     .reset_index()
                     .sort_values('M4'))
sns.barplot(x="M4", y="percentage", hue="isFraud", data=train_m4)
plt.title('TrainM4')
plt.subplot(3,3,5)
train_m5 = (train.groupby(['isFraud'])['M5']
                     .value_counts(normalize=True)
                     .rename('percentage')
                     .mul(100)
                     .reset_index()
                     .sort_values('M5'))
sns.barplot(x="M5", y="percentage", hue="isFraud", data=train_m5)
plt.title('TrainM5')
plt.subplot(3,3,6)
train_m6 = (train.groupby(['isFraud'])['M6']
                     .value_counts(normalize=True)
                     .rename('percentage')
                     .mul(100)
                     .reset_index()
                     .sort_values('M6'))
sns.barplot(x="M6", y="percentage", hue="isFraud", data=train_m6)
plt.title('TrainM6')
plt.subplot(3,3,7)
train_m7 = (train.groupby(['isFraud'])['M7']
                     .value_counts(normalize=True)
                     .rename('percentage')
                     .mul(100)
                     .reset_index()
                     .sort_values('M7'))
sns.barplot(x="M7", y="percentage", hue="isFraud", data=train_m7)
plt.title('TrainM7')
plt.subplot(3,3,8)
train_m8 = (train.groupby(['isFraud'])['M8']
                     .value_counts(normalize=True)
                     .rename('percentage')
                     .mul(100)
                     .reset_index()
                     .sort_values('M8'))
sns.barplot(x="M8", y="percentage", hue="isFraud", data=train_m8)
plt.title('TrainM8')
plt.subplot(3,3,9)
train_m9 = (train.groupby(['isFraud'])['M9']
                     .value_counts(normalize=True)
                     .rename('percentage')
                     .mul(100)
                     .reset_index()
                     .sort_values('M9'))
sns.barplot(x="M9", y="percentage", hue="isFraud", data=train_m9)
plt.title('TrainM9')

**M1 và M7 không có đóng góp gì trong việc phát hiện xem một giao dịch có gian lận và hợp pháp hay không.**

<h4> Phân tích V features </h4>

Lượng V_xxx quá nhiều -> khó phân tích và trực quan kĩ, nhưng em chỉ nắm được loại dữ liệu là float và nhiều giá trị NaN.

<h4> Phân tích DeviceInfo</h4>

Mô tả thiết bị người dùng giao dịch.

In [ ]:
train['DeviceInfo'].value_counts()[:10]

- Đối với mỗi thiết bị có nhiều model có thể có từ model cũ đến model mới.
- Có thể nhóm tất cả các thiết bị tương tự và tạo một tính năng khác.

In [ ]:
def transform_DeviceInfo(df):
    df['DeviceCorp'] = df['DeviceInfo']
    df.loc[df['DeviceInfo'].str.contains('HUAWEI|HONOR', case=False, na=False, regex=True), 'DeviceCorp'] = 'HUAWEI'
    df.loc[df['DeviceInfo'].str.contains('OS', na=False, regex=False), 'DeviceCorp'] = 'APPLE'
    df.loc[df['DeviceInfo'].str.contains('Idea|TA', case=False, na=False), 'DeviceCorp'] = 'Lenovo'
    df.loc[df['DeviceInfo'].str.contains('Moto|XT|Edison', case=False, na=False), 'DeviceCorp'] = 'Moto'
    df.loc[df['DeviceInfo'].str.contains('MI|Mi|Redmi', na=False), 'DeviceCorp'] = 'Mi'
    df.loc[df['DeviceInfo'].str.contains('VS|LG|EGO', na=False), 'DeviceCorp'] = 'LG'
    df.loc[df['DeviceInfo'].str.contains('ONE TOUCH|ALCATEL', case=False, na=False, regex=False), 'DeviceCorp'] = 'ALCATEL'
    df.loc[df['DeviceInfo'].str.contains('ONE A', na=False, regex=False), 'DeviceCorp'] = 'ONEPLUS'
    df.loc[df['DeviceInfo'].str.contains('OPR6', na=False, regex=False), 'DeviceCorp'] = 'HTC'
    df.loc[df['DeviceInfo'].str.contains('Nexus|Pixel', case=False, na=False, regex=True), 'DeviceCorp'] = 'google'
    df.loc[df['DeviceInfo'].str.contains('STV', na=False, regex=False), 'DeviceCorp'] = 'blackberry'
    df.loc[df['DeviceInfo'].str.contains('ASUS', case=False, na=False, regex=False), 'DeviceCorp'] = 'ASUS'
    df.loc[df['DeviceInfo'].str.contains('BLADE', case=False, na=False, regex=False), 'DeviceCorp'] = 'ZTE'
    
    df['DeviceCorp'] = df['DeviceInfo'].astype('str').str.split(':', expand=True)[0].\
                                str.split('-', expand=True)[0].str.split(expand=True)[0]
    
    df.loc[df['DeviceInfo'].isin(['rv', 'SM', 'GT', 'SGH']), 'DeviceCorp'] = 'SAMSUNG'
    df.loc[df['DeviceInfo'].str.startswith('Z', na=False), 'DeviceCorp'] = 'ZTE'
    df.loc[df['DeviceInfo'].str.startswith('KF', na=False), 'DeviceCorp'] = 'Amazon'
    
    for i in ['D', 'E', 'F', 'G']:
        df.loc[df['DeviceInfo'].str.startswith(i, na=False), 'DeviceCorp'] = 'SONY'

    df.loc[df['DeviceCorp'].isin(df['DeviceCorp'].value_counts()\
                                 [df['DeviceCorp'].value_counts() < 100].index), 'DeviceCorp'] = 'Other'
    df['DeviceCorp'] = df['DeviceCorp'].str.upper()
    
    return df

In [ ]:
train = transform_DeviceInfo(train)

In [ ]:
test = transform_DeviceInfo(test)

In [ ]:
def settingNaN(value):
    if value=='NAN':
        return np.nan
    else:
        return value

In [ ]:
train['DeviceCorp'] = train['DeviceCorp'].apply(settingNaN)
test['DeviceCorp'] = test['DeviceCorp'].apply(settingNaN)

In [ ]:
plt.figure(figsize=(11,6))
plt.subplot(1,2,1)
DeviceTypetrain = (train.groupby(['isFraud'])['DeviceType']
                     .value_counts(normalize=True)
                     .rename('percentage')
                     .mul(100)
                     .reset_index()
                     .sort_values('DeviceType'))
sns.barplot(x="DeviceType", y="percentage", hue="isFraud", data=DeviceTypetrain)
plt.title('TrainDeviceType')
plt.subplot(1,2,2)
DeviceTypeTest =test[~test['DeviceType'].isnull()]['DeviceType'].value_counts(normalize=True).mul(100).rename('percentage')\
.reset_index()
sns.barplot(x="index", y="percentage", data=DeviceTypeTest)
plt.title('TestDeviceType')

**Các danh mục trùng lặp cao, máy tính để bàn và điện thoại di động đều có cùng số lượng giao dịch hợp pháp và gian lận.**

<h4> Phân tích id_31 features</h4>

Mô tả version của trình duyệt của người dùng.

In [ ]:
train['id_31'].value_counts()[:10]

In [ ]:
def id31_split(dataframe):
    dataframe['browser_id_31'] = dataframe['id_31'].str.split(' ', expand=True)[0]
    dataframe['version_id_31'] = dataframe['id_31'].str.split(' ', expand=True)[1]

In [ ]:
id31_split(train)

In [ ]:
def id31_split_test(dataframe):
    dataframe['browser_id_31'] = dataframe['id-31'].str.split(' ', expand=True)[0]
    dataframe['version_id_31'] = dataframe['id-31'].str.split(' ', expand=True)[1]

In [ ]:
id31_split_test(test)

<h4> Phân tích id_30 features</h4>

Mô tả version OS của người dùng


In [ ]:
train['id_30'].value_counts()[:10]

In [ ]:
test['id-30'].value_counts()[:10]

- Id_30 represents the os  and its version. 

In [ ]:
def id30_split(dataframe):
    dataframe['OS_id_30'] = dataframe['id_30'].str.split(' ', expand=True)[0]
    dataframe['version_id_30'] = dataframe['id_30'].str.split(' ', expand=True)[1]

In [ ]:
id30_split(train)

In [ ]:
test['OS_id_30'] = test['id-30'].str.split(' ', expand=True)[0]
test['version_id_30'] = test['id-30'].str.split(' ', expand=True)[1]

<h4> Phân tích id_33 features</h4>

Mô tả độ rộng màn hình.

In [ ]:
train['id_33'].value_counts()[:10]

In [ ]:
test['id-33'].value_counts()[:10]

In [ ]:
def id33_split(dataframe):
    dataframe['screen_width'] = dataframe['id_33'].str.split('x', expand=True)[0]
    dataframe['screen_height'] = dataframe['id_33'].str.split('x', expand=True)[1]

In [ ]:
id33_split(train)

In [ ]:
test['screen_width'] = test['id-33'].str.split('x', expand=True)[0]
test['screen_height'] = test['id-33'].str.split('x', expand=True)[1]

#### Feature engineering

In [ ]:
def afterDecimalCount(amt):
    amtString = str(amt)
    return len(amtString.split(".")[1])

In [ ]:
train['TransDecimalCount'] = train['TransactionAmt'].apply(afterDecimalCount)
test['TransDecimalCount'] = test['TransactionAmt'].apply(afterDecimalCount)

In [ ]:
train.groupby("isFraud").mean()['TransDecimalCount']

Thực hiện số tương tác các features:

In [ ]:
train['card1_count_full'] = train['card1'].map(pd.concat([train['card1'], test['card1']], ignore_index=True).value_counts(dropna=False))
test['card1_count_full'] = test['card1'].map(pd.concat([train['card1'], test['card1']], ignore_index=True).value_counts(dropna=False))
train['card2_count_full'] = train['card2'].map(pd.concat([train['card2'], test['card2']], ignore_index=True).value_counts(dropna=False))
test['card2_count_full'] = test['card2'].map(pd.concat([train['card2'], test['card2']], ignore_index=True).value_counts(dropna=False))
train['card3_count_full'] = train['card3'].map(pd.concat([train['card3'], test['card3']], ignore_index=True).value_counts(dropna=False))
test['card3_count_full'] = test['card3'].map(pd.concat([train['card3'], test['card3']], ignore_index=True).value_counts(dropna=False))
train['card4_count_full'] = train['card4'].map(pd.concat([train['card4'], test['card4']], ignore_index=True).value_counts(dropna=False))
test['card4_count_full'] = test['card4'].map(pd.concat([train['card4'], test['card4']], ignore_index=True).value_counts(dropna=False))
train['card5_count_full'] = train['card5'].map(pd.concat([train['card5'], test['card5']], ignore_index=True).value_counts(dropna=False))
test['card5_count_full'] = test['card5'].map(pd.concat([train['card5'], test['card5']], ignore_index=True).value_counts(dropna=False))
train['card6_count_full'] = train['card6'].map(pd.concat([train['card6'], test['card6']], ignore_index=True).value_counts(dropna=False))
test['card6_count_full'] = test['card6'].map(pd.concat([train['card6'], test['card6']], ignore_index=True).value_counts(dropna=False))
train['addr1_count_full'] = train['addr1'].map(pd.concat([train['addr1'], test['addr1']], ignore_index=True).value_counts(dropna=False))
test['addr1_count_full'] = test['addr1'].map(pd.concat([train['addr1'], test['addr1']], ignore_index=True).value_counts(dropna=False))
train['addr2_count_full'] = train['addr2'].map(pd.concat([train['addr2'], test['addr2']], ignore_index=True).value_counts(dropna=False))
test['addr2_count_full'] = test['addr2'].map(pd.concat([train['addr2'], test['addr2']], ignore_index=True).value_counts(dropna=False))
train['TransactionAmt_to_mean_card1'] = train['TransactionAmt'] / train.groupby(['card1'])['TransactionAmt'].transform('mean')
train['TransactionAmt_to_mean_card4'] = train['TransactionAmt'] / train.groupby(['card4'])['TransactionAmt'].transform('mean')
train['TransactionAmt_to_std_card1'] = train['TransactionAmt'] / train.groupby(['card1'])['TransactionAmt'].transform('std')
train['TransactionAmt_to_std_card4'] = train['TransactionAmt'] / train.groupby(['card4'])['TransactionAmt'].transform('std')
test['TransactionAmt_to_mean_card1'] = test['TransactionAmt'] / test.groupby(['card1'])['TransactionAmt'].transform('mean')
test['TransactionAmt_to_mean_card4'] = test['TransactionAmt'] / test.groupby(['card4'])['TransactionAmt'].transform('mean')
test['TransactionAmt_to_std_card1'] = test['TransactionAmt'] / test.groupby(['card1'])['TransactionAmt'].transform('std')
test['TransactionAmt_to_std_card4'] = test['TransactionAmt'] / test.groupby(['card4'])['TransactionAmt'].transform('std')
train['id_02_to_mean_card1'] = train['id_02'] / train.groupby(['card1'])['id_02'].transform('mean')
train['id_02_to_mean_card4'] = train['id_02'] / train.groupby(['card4'])['id_02'].transform('mean')
train['id_02_to_std_card1'] = train['id_02'] / train.groupby(['card1'])['id_02'].transform('std')
train['id_02_to_std_card4'] = train['id_02'] / train.groupby(['card4'])['id_02'].transform('std')
test['id_02_to_mean_card1'] = test['id-02'] / test.groupby(['card1'])['id-02'].transform('mean')
test['id_02_to_mean_card4'] = test['id-02'] / test.groupby(['card4'])['id-02'].transform('mean')
test['id_02_to_std_card1'] = test['id-02'] / test.groupby(['card1'])['id-02'].transform('std')
test['id_02_to_std_card4'] = test['id-02'] / test.groupby(['card4'])['id-02'].transform('std')
train['D15_to_mean_card1'] = train['D15'] / train.groupby(['card1'])['D15'].transform('mean')
train['D15_to_mean_card4'] = train['D15'] / train.groupby(['card4'])['D15'].transform('mean')
train['D15_to_std_card1'] = train['D15'] / train.groupby(['card1'])['D15'].transform('std')
train['D15_to_std_card4'] = train['D15'] / train.groupby(['card4'])['D15'].transform('std')
test['D15_to_mean_card1'] = test['D15'] / test.groupby(['card1'])['D15'].transform('mean')
test['D15_to_mean_card4'] = test['D15'] / test.groupby(['card4'])['D15'].transform('mean')
test['D15_to_std_card1'] = test['D15'] / test.groupby(['card1'])['D15'].transform('std')
test['D15_to_std_card4'] = test['D15'] / test.groupby(['card4'])['D15'].transform('std')
train['D15_to_mean_addr1'] = train['D15'] / train.groupby(['addr1'])['D15'].transform('mean')
train['D15_to_mean_card4'] = train['D15'] / train.groupby(['card4'])['D15'].transform('mean')
train['D15_to_std_addr1'] = train['D15'] / train.groupby(['addr1'])['D15'].transform('std')
train['D15_to_std_card4'] = train['D15'] / train.groupby(['card4'])['D15'].transform('std')

test['D15_to_mean_addr1'] = test['D15'] / test.groupby(['addr1'])['D15'].transform('mean')
test['D15_to_mean_card4'] = test['D15'] / test.groupby(['card4'])['D15'].transform('mean')
test['D15_to_std_addr1'] = test['D15'] / test.groupby(['addr1'])['D15'].transform('std')

test['D15_to_std_card4'] = test['D15'] / test.groupby(['card4'])['D15'].transform('std')

**Tiếp cận với các feature V, loại bỏ những dữ liệu dư thừa:**

In [ ]:
vfeatures = []
for i in train.columns:
    if 'V' in i:
        vfeatures.append(i)

In [ ]:
vfeatures.remove('card3Values')
vfeatures.append('isFraud')

**Chia data thành 70:30 để train**

In [ ]:
v_train = train[:412785]
v_cv = train[412785:]

In [ ]:
v_train = v_train[vfeatures]
v_cv = v_cv[vfeatures]

In [ ]:
v_train_x  = v_train.drop(['isFraud'],axis=1)
v_train_y = v_train['isFraud']
v_cv_x = v_cv.drop(['isFraud'],axis=1)
v_cv_y = v_cv['isFraud']

**Gán các giá trị V với giá trị trung bình:**

In [ ]:
v_train_x.fillna(v_train_x.mean(),inplace=True)
v_cv_x.fillna(v_cv_x.mean(),inplace=True)

Dùng LGBM để test (vì tốc độ nhanh):

In [ ]:
rf = LGBMClassifier()
rf.fit(v_train_x,v_train_y)
predict_y_=rf.predict_proba(v_train_x)
predict_y = rf.predict_proba(v_cv_x)
print('train auc:',roc_auc_score(v_train_y,predict_y_[:,1]))
print('cv auc:',roc_auc_score(v_cv_y,predict_y[:,1]))

**Loại bỏ feature dựa vào độ quan trọng (importance):**

In [ ]:
g = rf.feature_importances_
j=1
vremove=[]
for i in g:
    if i<2:
        vremove.append('V'+str(j))
    j+=1
len(vremove)

In [ ]:
for i in vremove:
    vfeatures.remove(i)
    print('removed:',i)

In [ ]:
v_train = v_train[vfeatures]
v_cv = v_cv[vfeatures]

In [ ]:
v_train_x  = v_train.drop(['isFraud'],axis=1)
v_train_y = v_train['isFraud']
v_cv_x = v_cv.drop(['isFraud'],axis=1)
v_cv_y = v_cv['isFraud']

In [ ]:
v_train_x.fillna(v_train_x.mean(),inplace=True)
v_cv_x.fillna(v_cv_x.mean(),inplace=True)

In [ ]:
rf = LGBMClassifier()
rf.fit(v_train_x,v_train_y)
predict_y_=rf.predict_proba(v_train_x)
predict_y = rf.predict_proba(v_cv_x)
print('train auc:',roc_auc_score(v_train_y,predict_y_[:,1]))
print('cv auc:',roc_auc_score(v_cv_y,predict_y[:,1]))

In [ ]:
# train.to_csv('new_train_01.csv',index = False)

In [ ]:
# test.to_csv('new_test_01.csv',index = False)